
# Prepare ReCON Webcam Images for Optical Wave Gauging

This code was written by [Nicholas Catanzaro](https://github.com/ncata/GLERL_contract) for NOAA GLERL 

#### Note: 

##### Functionality
The function below is specifically for the analysis of view \#2 in MI City. This works with imagery from the old webcam location. It is designed for a proof of concept application of the OWG. This function will need to be adapted or rewritten to deal with contemporary datsets. Also, a record of the inputs to each version should be kept for research/repeatability purposes.

##### Next Steps
The most important additon that is needed is a way to automate the creation of view folders within the new year_imgprep folder. This could be done by including a list of numbers as strings representing the camera view as an input, or using string parsing and appending the target string-numbers into a list within the function. The latter option is likely best as it requires no additonal inputs be added to the funciton

In [47]:
import os
import shutil
from PIL import Image
import numpy as np
from distutils.dir_util import copy_tree
import matplotlib.pyplot as plt
import time

In [6]:
def estimate_sharpness(img): 
    """
    Estimate image sharpness 
    Input:
        img - np.ndarray representing image
              img read as skimage.io.imread( 'imagefilename.jpg' )
    Returns:
        s - sharpness estimates

    adapted from https://stackoverflow.com/questions/6646371/detect-which-image-is-sharper
    """
    gy, gx = np.gradient(img)
    gnorm = np.sqrt(gx**2 + gy**2)
    sharpness = np.average(gnorm)
    return sharpness
# edited to remove the part where they convert to grayscale as my images already are grayscale. Removed contrast because I do not need it

## Function version 2.0

In [86]:
def prepowgimgs(year, directory, arrayshape, resolution, trial, viewlist):
    '''Create a new directory for images in each view and run QC and primary augmentation before interpolation and then model 
    training, validation, or testing. version 2.0
    
    Year is the target year of images, 
    directory is where the year files are stored,
    resolution is the resolutiont the images will be reduced to, 
    trial is the name of the trial that these inputs are associate with. A record of trial details should be kept by hand and stored
    in C:/njc/src/SSF/OWG/OWG_records/trial. 
    viewlist is a list of the view numbers found in the target year folder
    '''
    start = time.time()
    #set counters 
    counter = 0
    nightcounter = 0
    blurcounter = 0
    failcounter = 0
    
    # create a new directory with the called year_imgprep 
    foldername = "/{}_imgprep".format(str(year))
    newdir = directory+foldername+"{}".format(trial)
    try:
        shutil.rmtree(newdir)
        print("Folder found: deleting contents")
        print("Creating folder")
        os.mkdir(newdir)
    except:
        print("Creating folder")
        os.mkdir(newdir)

    # copy files into new directory, this can take a while
    print("Moving Files")
    copy_tree(directory+"/{}".format(str(year)), newdir)
    
    # seperate the views into different folders within the directory
    viewdirs = []
    for view in viewlist:
        print("Seperating View {}".format(view))
        viewpath = newdir+"/view{}".format(view)
        viewdirs.append(viewpath)
        os.mkdir(viewpath) 
    
    
    for viewpath in viewdirs:
        print ("Moving images to {}".format(viewpath))
        # move images into views
        for filename in os.listdir(newdir):
            if filename[-5] == viewpath[-1]:
                os.rename(newdir+"/{}".format(filename), viewpath+"/{}".format(filename.replace(filename[13:16],"")))   
        
        # edit images from desired view into cropped lower resolution grayscale and store in new folder
        print ("Augmenting and filtering imgaes in {}".format(viewpath))
        for filename in os.listdir(viewpath):
            if filename.endswith(".jpg"):
                image = Image.open(viewpath+"/{}".format(filename)).convert("L")
            
                # select target portion of image to make it square
                bwarray = np.array(image) 
                small = bwarray[arrayshape]
                
                # change image resoltuion
                img = Image.fromarray(small)
                resize = (resolution, resolution)
                smaller = img.resize(resize)
               
                #Get QC metrics
                sharpness = estimate_sharpness(np.asarray(smaller))
                avgintensity = np.mean(np.asarray(smaller))
                
                # perform QC
                if sharpness < 3.5 or avgintensity < 85:
                    os.remove(viewpath+"/"+filename)
                    failcounter = failcounter + 1
                else:
                    # save images to OWG directory
                    try:
                        counter = counter + 1
                        owgimage = smaller.save(viewpath+"/{}".format(filename))
                    except IOError:
                        print("cannot create image for", filename)
                        failcounter = failcounter + 1
        
                    # remove the underscore in the filename
                    os.rename(viewpath+"/"+filename, viewpath+"/"+filename.replace("_",""))
                    
    print (failcounter, "images removed for quality control")
    print (counter, "images processed")
    end = time.time()   
    return print ("Completed in {} minutes.".format((end - start)/60))

#### Test function version 2.0

In [88]:
year = 2017
directory = "C:/njc/src/mcyimgs"
arrayshape = np.index_exp[100:700, 680:1280]
resolution = 512
trial = 'test'
viewlist = [1,2]
prepowgimgs(year, directory, arrayshape, resolution, trial, viewlist)

Folder found: deleting contents
Creating folder
Moving Files
Seperating View 1
Seperating View 2
Moving images to C:/njc/src/mcyimgs/2017_imgpreptest/view1
Augmenting and filtering imgaes in C:/njc/src/mcyimgs/2017_imgpreptest/view1
Moving images to C:/njc/src/mcyimgs/2017_imgpreptest/view2
Augmenting and filtering imgaes in C:/njc/src/mcyimgs/2017_imgpreptest/view2
5311 images removed for quality control
2098 images processed
Completed in 2.888287130991618 minutes.


### Select out random images from a year to use as validation data
csv's generated from functions in csv_interpolation notebook

In [1]:
import pandas as pd
df = pd.read_csv('C:/njc/src/SSF/OWG/mcyv22017trial3.csv', sep = ",", index_col=False)
df
# random sample ~30%
dfs = df.sample(n= int(.3*819))
dfs.reset_index(drop=True, inplace=True)
dfs
dfs.to_csv('C:/njc/src/SSF/OWG/mcyv22017(30trial3).csv', sep = ",")

### Drop rows of original dataframe if they have been randomly selected for validation
this creates the csv file that will be used to train the OWG

In [2]:
# load in the dataframes
total = pd.read_csv('C:/njc/src/SSF/OWG/mcyv22017trial3.csv', sep = ",", index_col=False)
validation = pd.read_csv('C:/njc/src/SSF/OWG/mcyv22017(30trial3).csv', sep = ",", index_col=False)

# if the imagename in the total dataframe exists in the validation dataframe do not include it in the train dataframe. 
# the ~ inverts the boolean indexing, which we need to do because only True values will be kept and they start as False as they are not in the validation dataframe
train = total[~total.id.isin(validation.id)]
train
train.to_csv('C:/njc/src/SSF/OWG/mcyv22017trial3train.csv', sep = ",")